In [1]:
import numpy as np
from sympy import symbols as syb
from sympy import MatrixSymbol as msyb
from sympy import sqrt, Array, I, Matrix

In [47]:
from sympy.physics.quantum import Bra,Ket, TensorProduct
from sympy.physics.quantum.gate import HadamardGate as H
from sympy.physics.quantum.gate import ZGate as Z
from sympy import simplify, Eq, linsolve, solve
from sympy.solvers import solve

- $|\psi(0)\rangle = \frac{1}{\sqrt{2E}} \begin{bmatrix} \sqrt{M}\dot{\vec{x}}(0) \\ i\vec{\mu}(0) \end{bmatrix} \text{ and  } |\psi(t)\rangle = \frac{1}{\sqrt{2E}} \begin{bmatrix} \sqrt{M}\dot{\vec{x}}(t) \\ i\vec{\mu}(t) \end{bmatrix} $ where $dim(|\psi\rangle) = N + M | M = N(N+1)/2$

- Say $n = \log_2 N$

- I have assumed that the relative phase in the state only differ by $k\pi$, where $k$ is an integer, which is a fair assumption, as velocities and positions are are real terms.

- Another assumption I made is that, the state is separable in Kinetic Energy and potential energy qubits, hence, I can apply projective measurement separately to both part. This again is justifiable as because of how the state encding remains throughout the simulation.

- For only getiing the velocities, I require Computational basis measurement of all KE qubits and $X^{\otimes n}$ basis measurement of KE qubits. Which makes the complexity as $$\mathcal{O}()+\mathcal{O}(2)$$ which is a constant; as compared to the complexity for traditional state tomography which will be $$\mathcal{O}()+\mathcal{O}(4^n).$$ Exponential in n



#### For a quantum state of arbitrary size

In [134]:
def dummy_state(N=2,E=1):
  M = [1]*N # restricted to all mass 1 which can later be generalized
  K = msyb('K', N, N,)
  x = list(syb('x0:%d'%N,real = True)) #unknowns postion vector
  x_dot = list(syb('v0:%d'%N,real = True)) #unknown velocity vector
  # N(N+1)/2 terms, potential energy part of the quantum state
  mu = []
  for i in range(N):
      for j in range(N):
          if j>i:
              mu.append((I*sqrt(K[i,j])*(x[i]-x[j]))/sqrt(2*E))
          elif j==i:
              mu.append((I*sqrt(K[j,j])*x[j])/sqrt(2*E))
  # N terms kinetic energy part of the quantum state
  y_dot = []
  for i in range(N):
    y_dot.append(x_dot[i]*sqrt(M[i])/sqrt(2*E))
    psi0 = y_dot+mu
  #padding with zeros
  for i in range(int(3*N*(N-1)/2)):
    psi0.append(0)
  return psi0

def __get_ke_part(psi,N):
  return psi[:N]     #extracted the kitentic part of the state

def x_measure_equations(psi,N):
  psi0_ext = __get_ke_part(psi,N) #get the KE part
  q = int(np.log2(N))    #number of qubits carrying kinetic energy terms
  gate = H(q)
  H_matrix = gate.get_target_matrix()
  # Create the tensor product of the Hadamard gate for n qubits
  H_otimes_q = TensorProduct(*[H_matrix for _ in range(q)])
  phi0_ext = H_otimes_q*Matrix(psi0_ext)
  phi_amps = []
  for i in range(len(phi0_ext)):
    phi_amps.append(phi0_ext[i]*phi0_ext[i])
  return phi_amps

def post_process_ke(amps_list,out_probs_x,N):
  Eq_list = []
  for i in range(N):
    Eq_list.append(simplify(amps_list[i])-out_probs[i])
  solution = solve(Eq_list, x_dot,dict =True)
  return {"Equations":Eq_list, "Solutions": solution}

def eliminate_solution(out_probs_comp,solutions):
  filtered_sols = []
  for cases in solutions:
    # Check if all values in the dictionary match the corresponding compare value
    if np.allclose(out_probs_comp,[float(_) for _ in np.array(list(cases.values()))**2/2]):
      filtered_sols.append(cases)
  return filtered_sols

In [76]:
dummy_psi = dummy_state()
dummy_psi

[sqrt(2)*v0/2,
 sqrt(2)*v1/2,
 sqrt(2)*I*x0*sqrt(K[0, 0])/2,
 sqrt(2)*I*(x0 - x1)*sqrt(K[0, 1])/2,
 sqrt(2)*I*x1*sqrt(K[1, 1])/2,
 0,
 0,
 0]

In [77]:
amps_list = x_measure_equations(dummy_psi,2)

In [135]:
out_probs_x = [3/4,1/4]
out_probs_comp = [(2-np.sqrt(3))/4,(2+np.sqrt(3))/4]
post_process_out = post_process_ke(amps_list,out_probs_x,2)
eliminate_solution(out_probs_comp,post_process_out["Solutions"])

[{v0: -0.366025403784439, v1: -1.36602540378444},
 {v0: 0.366025403784439, v1: 1.36602540378444}]